# Semantic model security updates
This notebook helps to configure security on an existing semantic model. RLS roles can be scripted and added to the model. Use case could be to run this notebook regularly in case security may be overwritten by a semantic model update. Security roles currently only include RLS. OLS is not applied. 

<mark>**Note:**</mark> Some code is AI generated. Use on your own risk. 

In [1]:
# Import libraries
%pip install semantic-link-labs
import sempy.fabric as fabric
import sempy_labs as labs
from sempy_labs import tom

StatementMeta(, 5eb3616c-c8fe-4987-b6da-56b34bbf7985, 8, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 764.7/764.7 kB 99.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 204.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 kB 64.5 MB/s eta 0:00:00
  Attempting uninstall: azure-core
    Found existing installation: azure-core 1.30.2
    Not uninstalling azure-core at /home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages, outside environment /nfs4/pyenv-f6372dfc-a72b-4fe5-b980-458e3a5c9864
    Can't uninstall 'azure-core'. No files were found to uninstall.
  Attempting uninstall: semantic-link-sempy
    Found existing installation: semantic-link-sempy 0.11.0
    Not uninstalling semantic-link-sempy at /home/trusted-service-user/clust

### Specify semantic model to manage security

In [13]:
semanticmodel_name = "AW2020" # specify semantic model name or id
workspace_id = fabric.get_workspace_id() # Retrieves the workspace id in which the notebook is running

StatementMeta(, 5eb3616c-c8fe-4987-b6da-56b34bbf7985, 21, Finished, Available, Finished)

### Setting up TOM connection
The TOM (Tabular Object Model) wrapper is used to make advanced changes to the semantic model definition

In [14]:
# Create a read/write TOM connection
# This cell takes the semantic model definition in memory
tw = tom.TOMWrapper(
    dataset = semanticmodel_name,
    workspace = workspace_id,
    readonly = False
)

StatementMeta(, 5eb3616c-c8fe-4987-b6da-56b34bbf7985, 22, Finished, Available, Finished)

#### Define role setup
This section defines the role to create / maintain and role memberships

In [16]:
rolename = "test" # Name of the security role in the model 
roledescription = "test description" # description for the security role in the model 
members = ["user1@example.com", "user2@example.com", "user3@example.com"] # list of users to add 
member_type = "User" # User or Group (in case of Entra security groups)

StatementMeta(, 5eb3616c-c8fe-4987-b6da-56b34bbf7985, 24, Finished, Available, Finished)

In [17]:
# Create role if not existing yet
try:
    tw.add_role(
        role_name = rolename,
        model_permission = None, # Defaults to read
        description = roledescription
    )
    print(f"✅ Role '{rolename}' successfully created.")
except Exception as e:
    if "already exists" in str(e):
        print(f"⚠️ Role '{rolename}' already exists — continuing with members and RLS.")
    else:
        raise

StatementMeta(, 5eb3616c-c8fe-4987-b6da-56b34bbf7985, 25, Finished, Available, Finished)

⚠️ Role 'test' already exists — continuing with members and RLS.


In [ ]:
# Add each member (skip existing ones)
for member in members:
    try:
        tw.add_role_member(
            role_name = rolename,
            member = member,
            role_member_type = member_type
        )
        print(f"👤 Added member '{member}' to role '{rolename}'.")
    except Exception as e:
        # catch any duplicate membership errors
        if "already exists in the collection" in str(e) or "already part of role" in str(e):
            print(f"ℹ️ Member '{member}' already part of role '{rolename}', skipping.")
        else:
            raise

#### Define security filter(s)
This section defines the filters applied to the (before) created role. This section only allows RLS roles (OLS not included (yet))

<mark>**Note:**</mark> In this setup there is an assumption that the semantic model will have relationships defined between tables which will ensure RLS on other tables too. It is up to your own responsibility to check if this is actually true and if any relationships must be modified to allow security filters to cross relationships set to both directions. 

In [22]:
table = "DimProduct"
expression = 'DimProduct[Color] = "Blue"'

StatementMeta(, 5eb3616c-c8fe-4987-b6da-56b34bbf7985, 30, Finished, Available, Finished)

In [23]:
# Set or update RLS
try:
    tw.set_rls(
        role_name = rolename,
        table_name = table,
        filter_expression = expression
    )
    print(f"🔒 RLS filter set for role '{rolename}'.")
except Exception as e:
    print(f"⚠️ Failed to set RLS filter: {e}")


StatementMeta(, 5eb3616c-c8fe-4987-b6da-56b34bbf7985, 31, Finished, Available, Finished)

🔒 RLS filter set for role 'test'.


### Apply changes

In [24]:
# This step saves the changes back to the model in the workspace
tw.model.SaveChanges()

StatementMeta(, 5eb3616c-c8fe-4987-b6da-56b34bbf7985, 32, Finished, Available, Finished)